In [1]:
import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt
from keras import backend as K
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate,Permute,Reshape,Cropping2D,Maximum,Cropping1D
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
import fr_utils
from keras.layers.core import Lambda, Flatten, Dense
from theano.tensor import _shared
from keras.utils import to_categorical

C:\Users\ningj\AppData\Local\conda\conda\envs\bitch\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [15]:
import pickle
import os
f = open('train_set_new.pkl','rb+')
train_set =pickle.load(f)


In [2]:
def conv2d_bn(x, filters, kernel_size, strides=1, padding='same', activation='relu', use_bias=False, name=None):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias, kernel_initializer='he_normal', name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = None if name is None else name + '_bn'
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    return x

def mfm(x):
    shape = K.int_shape(x)
    x = Permute(dims=(3, 2, 1))(x) # swap 1 <-> 3 axis
    x1 = Cropping2D(cropping=((0, shape[3] // 2), 0))(x)
    x2 = Cropping2D(cropping=((shape[3] // 2, 0), 0))(x)
    x = Maximum()([x1, x2])
    x = Permute(dims=(3, 2, 1))(x) # swap 1 <-> 3 axis
    x = Reshape([shape[1], shape[2], shape[3] // 2])(x)
    return x

def common_conv2d(net, filters, filters2):
    res = net

    net = conv2d_bn(net, filters=filters, kernel_size=1, strides=1, padding='same')
    net = mfm(net)
    net = conv2d_bn(net, filters=filters2, kernel_size=3, strides=1, padding='same')
    net = mfm(net)

    return net

def lcnn9(inputs):
    # Conv1
    net = conv2d_bn(inputs, filters=96, kernel_size=5, strides=1, padding='same')
    net = mfm(net)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)

    # Block1
    net = common_conv2d(net,filters=96, filters2=192)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)

    # Block2
    net = common_conv2d(net,filters=192, filters2=384)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)

    # Block3
    net = common_conv2d(net,filters=384, filters2=256)

    # Block4
    net = common_conv2d(net,filters=256, filters2=256)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)

    net = Flatten()(net)

    return net


In [41]:
input_image = Input(shape=(250, 250, 3))

lcnn_output = lcnn9(inputs=input_image)

fc1 = Dense(512, activation=None)(lcnn_output)
fc1 = Reshape((512, 1))(fc1)
fc1_1 = Cropping1D(cropping=(0, 256))(fc1)
fc1_2 = Cropping1D(cropping=(256, 0))(fc1)
fc1 = Maximum()([fc1_1, fc1_2])
fc1 = Flatten()(fc1)

out = Dense(18, activation='softmax')(fc1)
model = Model(inputs=[input_image], outputs=out)

In [42]:
y=np.ones((3,1),dtype='int')
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train[:32],to_categorical(y_train[:32],18),epoch=5,batch_size=8)

C:\Users\ningj\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/5
32/32 [==============================] - 155s 5s/step - loss: 12.0997 - acc: 0.0625
Epoch 2/5
32/32 [==============================] - 145s 5s/step - loss: 13.5996 - acc: 0.1562
Epoch 3/5
32/32 [==============================] - 148s 5s/step - loss: 13.5996 - acc: 0.1562
Epoch 4/5
32/32 [==============================] - 144s 5s/step - loss: 13.5996 - acc: 0.1562
Epoch 5/5


KeyboardInterrupt: 

In [16]:
x_train=[]
y_train=[]
label=[]
num=0
k=list(range(5749))
for key,val in train_set[0].items():
    label.append(key)
    y_train.append(np.asarray([num]*val.shape[0]))
    num+=1
    x_train.append(val)

In [17]:
x_train=np.asarray(x_train)
y_train=np.asarray(y_train).reshape(610,1)

labels=dict(zip(k,label))

In [18]:
print(y_train[:32].shape)

(32, 1)


In [19]:
m=open('train_set.pkl','rb+')
dd=pickle.load(m)

In [20]:
re_label=dict(zip(label,k))
re_label

{'Aaron_Peirsol': 0,
 'Abdoulaye_Wade': 1,
 'Abdullah': 2,
 'Abdullah_Gul': 3,
 'Abel_Pacheco': 4,
 'Adam_Sandler': 5,
 'Adrien_Brody': 6,
 'Ahmed_Chalabi': 7,
 'Ai_Sugiyama': 8,
 'Al_Gore': 21,
 'Al_Sharpton': 22,
 'Alan_Greenspan': 9,
 'Alastair_Campbell': 10,
 'Albert_Costa': 11,
 'Alec_Baldwin': 12,
 'Alejandro_Toledo': 13,
 'Aleksander_Kwasniewski': 14,
 'Alexander_Downer': 15,
 'Alexander_Losyukov': 16,
 'Ali_Naimi': 17,
 'Allyson_Felix': 18,
 'Alvaro_Silva_Calderon': 19,
 'Alvaro_Uribe': 20,
 'Amanda_Bynes': 23,
 'Amelia_Vega': 24,
 'Amelie_Mauresmo': 25,
 'Amer_al-Saadi': 26,
 'Ana_Guevara': 27,
 'Ana_Palacio': 28,
 'Anders_Fogh_Rasmussen': 29,
 'Andre_Agassi': 30,
 'Andy_Roddick': 31,
 'Angela_Bassett': 32,
 'Angela_Merkel': 33,
 'Angelina_Jolie': 34,
 'Angelo_Reyes': 35,
 'Ann_Veneman': 37,
 'Anna_Kournikova': 36,
 'Antonio_Banderas': 38,
 'Antonio_Palocci': 39,
 'Antony_Leung': 40,
 'Ari_Fleischer': 43,
 'Arianna_Huffington': 41,
 'Ariel_Sharon': 42,
 'Arminio_Fraga': 44,
 '

In [21]:
x_train=[]
y_train=[]
for i in range(len(dd)):
    x_train.append(dd[i]['image'])
    y_train.append(re_label[dd[i]['label']])
x_train=np.asarray(x_train)
y_train=np.asarray(y_train).reshape(len(dd),1)

In [22]:
print(y_train.shape)

(6123, 1)


In [ ]:
model.predict(x_train[2:3])

In [23]:
labels

{0: 'Aaron_Peirsol',
 1: 'Abdoulaye_Wade',
 2: 'Abdullah',
 3: 'Abdullah_Gul',
 4: 'Abel_Pacheco',
 5: 'Adam_Sandler',
 6: 'Adrien_Brody',
 7: 'Ahmed_Chalabi',
 8: 'Ai_Sugiyama',
 9: 'Alan_Greenspan',
 10: 'Alastair_Campbell',
 11: 'Albert_Costa',
 12: 'Alec_Baldwin',
 13: 'Alejandro_Toledo',
 14: 'Aleksander_Kwasniewski',
 15: 'Alexander_Downer',
 16: 'Alexander_Losyukov',
 17: 'Ali_Naimi',
 18: 'Allyson_Felix',
 19: 'Alvaro_Silva_Calderon',
 20: 'Alvaro_Uribe',
 21: 'Al_Gore',
 22: 'Al_Sharpton',
 23: 'Amanda_Bynes',
 24: 'Amelia_Vega',
 25: 'Amelie_Mauresmo',
 26: 'Amer_al-Saadi',
 27: 'Ana_Guevara',
 28: 'Ana_Palacio',
 29: 'Anders_Fogh_Rasmussen',
 30: 'Andre_Agassi',
 31: 'Andy_Roddick',
 32: 'Angela_Bassett',
 33: 'Angela_Merkel',
 34: 'Angelina_Jolie',
 35: 'Angelo_Reyes',
 36: 'Anna_Kournikova',
 37: 'Ann_Veneman',
 38: 'Antonio_Banderas',
 39: 'Antonio_Palocci',
 40: 'Antony_Leung',
 41: 'Arianna_Huffington',
 42: 'Ariel_Sharon',
 43: 'Ari_Fleischer',
 44: 'Arminio_Fraga',
 4

In [27]:
import cv2
cv2.imwrite('test.jpg',x_test)

array([[0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [4],
       [4],
       [4],
       [5],
       [5]])

In [25]:
pickle.dump([x_train],open('x_train.pkl','wb'))
pickle.dump([y_train],open('y_train.pkl','wb'))

In [26]:
pickle.dump([labels,re_label],open('labels.pkl','wb'))